In [ ]:
import psycopg2
import geopandas as gpd
import folium
from IPython.display import display

# --- Database connection info ---
POSTGRES_HOST = "localhost"
POSTGRES_PORT = 5432
POSTGRES_DB = "osm"
POSTGRES_USER = "osmuser"
POSTGRES_PASSWORD = "N123456n"

def query_to_gdf(sql):
    conn = psycopg2.connect(
        dbname=POSTGRES_DB,
        user=POSTGRES_USER,
        password=POSTGRES_PASSWORD,
        host=POSTGRES_HOST,
        port=POSTGRES_PORT
    )
    gdf = gpd.read_postgis(sql, conn, geom_col='way')
    conn.close()
    return gdf

# --- SQL to fetch all UAE regions ---
uae_sql = """
 
SELECT
  name,
  boundary,
  admin_level,
  way,tags
FROM
  planet_osm_polygon
WHERE
  boundary = 'administrative'
 ;
"""

# --- Load regions ---
uae_gdf = query_to_gdf(uae_sql)
uae_gdf
if uae_gdf.empty:
    print("No UAE regions found. Check data import or query.")
else:
    print(f"Found {len(uae_gdf)} UAE region(s)")
    display(uae_gdf)

    # --- Reproject to EPSG:4326 ---
    uae_gdf = uae_gdf.to_crs(epsg=4326)

    # --- Save GeoJSON ---
    geojson_path = "uae_regions.geojson"
    uae_gdf.to_file(geojson_path, driver="GeoJSON")
    print(f"GeoJSON saved to {geojson_path}")

    # --- Visualize with Folium ---
    centroid = uae_gdf.unary_union.centroid
    m = folium.Map(location=[centroid.y, centroid.x], zoom_start=7)
    folium.GeoJson(uae_gdf, name="UAE Regions").add_to(m)
    display(m)


/home/noaman/Desktop/work/postgis/.venv/lib/python3.12/site-packages/geopandas/io/sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


Found 4619 UAE region(s)


/home/noaman/Desktop/work/postgis/.venv/lib/python3.12/site-packages/geopandas/io/sql.py:466: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(spatial_ref_sys_sql, con)


,name,boundary,admin_level,way,tags
0,الإمارات العربية المتحدة,administrative,2,"POLYGON ((6062064.852 2912559.522, 6062450.786...","""flag""=>""https://upload.wikimedia.org/wikipedi..."
1,جَزِيرَة صَلاَحَة,administrative,8,"POLYGON ((5955753.469 2776453.661, 5955765.102...","""name:ar""=>""جَزِيرَة صَلاَحَة"", ""name:en""=>""Ja..."
2,المرْفَأ,administrative,8,"POLYGON ((5944364.216 2761314.442, 5944515.756...","""name:ar""=>""المرْفَأ"", ""name:en""=>""Al Marfa"""
3,الهَرميَّة,administrative,8,"POLYGON ((5933052.486 2766068.775, 5933063.14 ...","""name:ar""=>""الهَرميَّة"", ""name:en""=>""Al Harmiy..."
4,جزيرة جنانة,administrative,8,"POLYGON ((5940090.973 2779205.361, 5940157.92 ...","""name:ar""=>""جزيرة جنانة"", ""name:en""=>""Jinanah ..."
...,...,...,...,...,...
4614,The Meadows 8,administrative,10,"POLYGON ((6142671.006 2883601.659, 6142673.099...","""name:en""=>""The Meadows 8"""
4615,Clubhouse Area,administrative,10,"POLYGON ((6141078.013 2885491.481, 6141133.127...","""name:en""=>""Clubhouse Area"""
4616,ميدوز,administrative,8,"POLYGON ((6138878.663 2884423.691, 6139044.395...","""name:ar""=>""ميدوز"", ""name:en""=>""The Meadows"""
4617,ميدوز 4,administrative,10,"POLYGON ((6139794.811 2884008.989, 6139795.868...","""name:ar""=>""ميدوز 4"", ""name:en""=>""Meadows 4"""


GeoJSON saved to uae_regions.geojson


/tmp/ipykernel_44163/3393510301.py:58: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = uae_gdf.unary_union.centroid


In [21]:
uae_gdf.head()


,name,name_en,boundary,admin_level,way,tags
0,جنوب مدينه سعد العبدالله,South of Saad Al-Abdullah City,administrative,6,"POLYGON ((47.62582 29.22489, 47.62583 29.224, ...","""name:ar""=>""جنوب مدينه سعد العبدالله"", ""name:e..."
1,المنطقة الشرقية,Eastern Province,administrative,4,"POLYGON ((48.86032 27.60599, 48.86078 27.60497...","""name:ar""=>""المنطقة الشرقية"", ""name:be""=>""Эш-Ш..."
2,محافظة الخفجي,Al Khafji Governorate,administrative,6,"POLYGON ((47.55689 28.13457, 47.55721 28.13291...","""name:ar""=>""محافظة الخفجي"", ""name:en""=>""Al Kha..."
3,محافظة الخفجي,Al Khafji Governorate,administrative,6,"POLYGON ((48.62914 28.11167, 48.62916 28.11163...","""name:ar""=>""محافظة الخفجي"", ""name:en""=>""Al Kha..."
4,المنطقة الشرقية,Eastern Province,administrative,4,"POLYGON ((48.62914 28.11167, 48.62916 28.11163...","""name:ar""=>""المنطقة الشرقية"", ""name:be""=>""Эш-Ш..."


In [11]:
import psycopg2
import geopandas as gpd
import folium
from IPython.display import display

# --- Database connection info ---
POSTGRES_HOST = "localhost"
POSTGRES_PORT = 5432
POSTGRES_DB = "osm"
POSTGRES_USER = "osmuser"
POSTGRES_PASSWORD = "N123456n"

def query_to_gdf(sql):
    conn = psycopg2.connect(
        dbname=POSTGRES_DB,
        user=POSTGRES_USER,
        password=POSTGRES_PASSWORD,
        host=POSTGRES_HOST,
        port=POSTGRES_PORT
    )
    gdf = gpd.read_postgis(sql, conn, geom_col='way')
    conn.close()
    return gdf

# --- SQL to fetch all UAE regions ---
uae_sql = """
SELECT
  name,
  boundary,
  admin_level,
  way
FROM
  planet_osm_polygon
WHERE
  boundary = 'administrative'
  AND name ILIKE ANY (ARRAY[
    '%abu dhabi%', 
    '%dubai%', 
    '%sharjah%', 
    '%ajman%', 
    '%umm al quwain%',
    '%ras al khaimah%',
    '%fujairah%'
  ])
  AND admin_level IN ('4', '5', '6', '8');
"""

# --- Load regions ---
uae_gdf = query_to_gdf(uae_sql)

if uae_gdf.empty:
    print("No UAE regions found. Check data import or query.")
else:
    print(f"Found {len(uae_gdf)} UAE region(s)")

    # Print all unique region names
    region_names = uae_gdf['name'].unique()
    print("UAE Regions:")
    for name in region_names:
        print(f" - {name}")

    # Optional: display table (works in Jupyter)
    display(uae_gdf[['name', 'admin_level', 'boundary']])

    # --- Reproject to EPSG:4326 ---
    uae_gdf = uae_gdf.to_crs(epsg=4326)

    # --- Save GeoJSON ---
    geojson_path = "uae_regions.geojson"
    uae_gdf.to_file(geojson_path, driver="GeoJSON")
    print(f"GeoJSON saved to {geojson_path}")

    # --- Visualize with Folium (just a map centered on UAE, no polygons) ---
    centroid = uae_gdf.unary_union.centroid
    m = folium.Map(location=[centroid.y, centroid.x], zoom_start=7)

    # If you want to show polygons, uncomment this line:
    # folium.GeoJson(uae_gdf, name="UAE Regions").add_to(m)

    display(m)


No UAE regions found. Check data import or query.


/home/noaman/Desktop/work/postgis/.venv/lib/python3.12/site-packages/geopandas/io/sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
